# Downloading needed ensemble CMIP6 data from model.variant entries in a .txt file

## Purpose of the notebook

This notebook aims at retrieving the wget script to download an ensemble of CMIP6 variables from a list of models.variant couples with all the criterias written in a *.csv* file. One example of such a file can be found in the repository. The main criterias, common to the whole ensemble, are defined **within this jupyter**.

It uses the *esgf-pyclient* library : https://esgf-pyclient.readthedocs.io/en/latest/index.html. Then it creates temporary folders to download the associated *.nc* files and moves these files to a more permanent storage folder. All the folders are cleaned up to insure that no mess is done. The path of the temporary downloading and the permanent storage folders are **user-defined**.

Feel free to share, use and improve the following code according to the provided license on the repository.

## Initialization 

### Set environment parameters

We set this parameter in order to avoid the display of needless and impactless errors regarding the *facets* variable.

In [1]:
# ================ SET ENVIRONMENT PARAMETERS ================ #

### NO FACETS WARNING ###

%env ESGF_PYCLIENT_NO_FACETS_STAR_WARNING = 42

env: ESGF_PYCLIENT_NO_FACETS_STAR_WARNING=42


### Importations

In [9]:
# ================ IMPORTATIONS ================ #

from pyesgf.search import SearchConnection  # the query package of pyesgf for cmip6 data

import os  # to handle path's management

import subprocess  # to run the wget code from python

import random # to generate the name of our downloading scripts

import pandas as pd # to retrieve the .csv data 

from library_for_download_notebooks import * # set of functions used in the notebook : to make it more readable

### Set the user-defined paths

These user defined paths are essential to make the program know 

* where it will create its **temporary download folder**
* where it will created its **permanent storage folder**
* where *research-criterias.csv* is located 

In [6]:
# ================ USER-DEFINED PATHS ================ #

### DEFINE TEMPORARY DOWNLOAD FOLDER'S PATH ###

download_dir = "/data/lgiboni/downloaded-cmip6-datadir-example"

### DEFINE PERMANENT STORAGE FOLDER'S PATH ###

storage_dir = "/scratchu/lgiboni/CMIP6_DATA"

### DEFINE LOCATION OF RESARCH-CRITERIAS.CSV ###

path_csv_file = "/home/lgiboni/CMIP6-Arctic-Aerosol-Analysis/utilities/download/research-criterias.csv"

## Extract the criterias from the .csv file 

### Open the .csv file in a pandas dataframe

In [12]:
# ================ OPEN THE CSV FILE ================ #

criterias = pd.read_csv(path_csv_file, delimiter = ";")

,project,variables,frequency,model,variant,experiment_id
0,CMIP6,"[""clt"",""rsdt"",""rsut"",""rsutcs"",""rsds"",""rsus"",""r...",mon,ACCESS-CM2,r1i1p1f1,"[""piClim-control, piClim-aer""]"
1,CMIP6,"[""clt"",""rsdt"",""rsut"",""rsutcs"",""rsds"",""rsus"",""r...",mon,ACCESS-ESM1-5,r1i1p1f1,"[""piClim-control, piClim-aer""]"
2,CMIP6,"[""clt"",""rsdt"",""rsut"",""rsutcs"",""rsds"",""rsus"",""r...",mon,BCC-ESM1,r1i1p1f1,"[""piClim-control, piClim-aer""]"
3,CMIP6,"[""clt"",""rsdt"",""rsut"",""rsutcs"",""rsds"",""rsus"",""r...",mon,CESM2,r1i1p1f1,"[""piClim-control, piClim-aer""]"
4,CMIP6,"[""clt"",""rsdt"",""rsut"",""rsutcs"",""rsds"",""rsus"",""r...",mon,CNRM-CM6-1,r1i1p1f2,"[""piClim-control, piClim-aer""]"
5,CMIP6,"[""clt"",""rsdt"",""rsut"",""rsutcs"",""rsds"",""rsus"",""r...",mon,CNRM-ESM2-1,r1i1p1f2,"[""piClim-control, piClim-aer""]"
6,CMIP6,"[""clt"",""rsdt"",""rsut"",""rsutcs"",""rsds"",""rsus"",""r...",mon,CanESM5,r1i1p2f1,"[""piClim-control, piClim-aer""]"
7,CMIP6,"[""clt"",""rsdt"",""rsut"",""rsutcs"",""rsds"",""rsus"",""r...",mon,GFDL-CM4,r1i1p1f1,"[""piClim-control, piClim-aer""]"
8,CMIP6,"[""clt"",""rsdt"",""rsut"",""rsutcs"",""rsds"",""rsus"",""r...",mon,GFDL-ESM4,r1i1p1f1,"[""piClim-control, piClim-aer""]"
9,CMIP6,"[""clt"",""rsdt"",""rsut"",""rsutcs"",""rsds"",""rsus"",""r...",mon,GISS-E2-1-G,r1i1p1f1,"[""piClim-control, piClim-aer""]"


We define all the specific information that are specific to our model.variant search as well as for the given experiment we are looking for.

In [7]:
# ================ DEFINE THE ADDITiONNAL SEARCH CRITERIAS FOR A GiVEN MODEL'S OUTPUT ================ #

### SOURCE ID : NAME OF THE CHOSEN MODEL ###

source_id = ["IPSL-CM6A-LR"]

### VARIANT OF THE MODEL RUN ###

variant_label = ["r1i1p1f1"]

### EXPERIMENT ID ###

experiment_id = ["piClim-control"]

### WHOLE SET OF QUERY PARAMETERS ###

facets = (
    "project,frequency,variable,source_id,variant_label,experiment_id,latest,replica"
)

*facets* will be used to confirm that our query holds all the parameters we are looking for.\
The element *latest* is to specify we want the latest data.\
The element *replica* is to specify whether we want duplicates to be included in our search results.

### Run the actual search

We start the search with the IPSL node. Since the variable *distrib* is set to **True**, we will extend the search to all the esgf nodes in case of lack of match.

In [8]:
# ================ RUN THE QUERY FOR THE PICLIM-CONTROL EXPERIMENT ================ #

### SET THE STARTING NODE FOR THE SEARCH ###

conn = SearchConnection("https://esgf-node.ipsl.upmc.fr/esg-search", distrib=True)

### QUERY ###

## Launch the query ##

query = conn.new_context(
    project=project,
    frequency=frequency,
    variable=variables,
    source_id=source_id,
    variant_label=variant_label,
    experiment_id=experiment_id,
    latest=True,  # Set to "True" to get the latest version
    replica=False,  # Set to "False" to avoid getting duplicate in the search results
    facets=facets,
)  # facets confirms the search criterias we have made


## Number of results ##

results_count = query.hit_count

## Print the number of results ##

print(f"The search has returned {results_count} results")

The search has returned 12 results


Since we have 12 variables, we have one result per variable for the researched model.variant couple and experiment.

## Create the folders for the downloading routine

Before saving the script and launching it, we need to make the folder associated to our given request. To do so, we create a temporary main folder for the downloading of the CMIP6 data and then make a subfolder for our given request.

### Make the temporary main folder for downloading the data

We create the folder cmip6-download-tmp at the path defined by the storage_dir variable thanks to the create_dir function.

In [9]:
# ================ MAKE THE MAIN TEMPORARY DOWNLOADING FOLDER ================ #

### USE OF THE CREATE DIR FUNCTION ###

temporary_downloading_folder_path = create_dir(storage_dir,  "cmip6-download-tmp")

Pre-existing folder at /scratchu/lgiboni/CMIP6_DATA/cmip6-download-tmp removed
New folder at /scratchu/lgiboni/CMIP6_DATA/cmip6-download-tmp created


### Make the temporary subfolders for our given request

We first define the name of the subfolders for our given request and call the create_dir function.

In [10]:
# ================ BUILD THE FOLDERS' NAME ================ #

### DEFINE THE SUBFOLDERS' PATH FOR OUR GIVEN REQUEST ###

## Define the model.variant folder's name ##

model_variant_name = source_id[0] + "." + variant_label[0]

## Define the model.variant/experiment subfolder's path ##

subfolder_request = model_variant_name + "/" + experiment_id[0]

### PRINT THE NAME ###

print("We will create {} at {}".format(subfolder_request, temporary_downloading_folder_path))

We will create IPSL-CM6A-LR.r1i1p1f1/piClim-control at /scratchu/lgiboni/CMIP6_DATA/cmip6-download-tmp


In [11]:
# ================ MAKE THE MAIN TEMPORARY DOWNLOADING FOLDER ================ #

### USE OF THE CREATE DIR FUNCTION ###

downloading_path_for_given_request = create_dir(temporary_downloading_folder_path,  subfolder_request)

New folder at /scratchu/lgiboni/CMIP6_DATA/cmip6-download-tmp/IPSL-CM6A-LR.r1i1p1f1/piClim-control created


## Launch the wget script for the request

Now that the folders have been created, we only have to write the wget script and download the data.

### Define its name and path

In [12]:
# ================ DEFINE ITS NAME AND PATH ================ #

### DEFINE ITS NAME ###

## Use of the random library to generate a unique downloading name

# Set the random number identifier #

random_id  = int(random.uniform(1,10)*1000) # we extract a random decimal number between 1 and 10 and keep the 4 first decimals

# Turn it into a str #

random_id = str(random_id)

## Define the downloading script's name ## 

downloading_script_name = "download-" + random_id + ".sh"

### DEFINE ITS PATH ###

downloading_script_path = downloading_path_for_given_request + "/" + downloading_script_name

### Write the downloading script

In [13]:
# ================ WRITE THE WGET SCRIPT ================ #

### RETRIEVE THE SCRIPT ###

wget_script_content = query.get_download_script()

### STORE IT IN THE DOWNLOADING FOLDER ###

## Write the wget script ##

with open(downloading_script_path, "w") as writer:

    # Writing it #
    
    writer.write(wget_script_content)

## Make it readable, writable and executable for the user ##

os.chmod(downloading_script_path, 0o750)  # it is also executable and readable by other users

### Run the downloading script 

In [14]:
# ================ RUN THE WGET SCRIPT ================ #

### RUN THE WGET SCRIPT ###

## We store the results in the download_process variable to produce a log ##

download_process = subprocess.run(
    "bash " + downloading_script_name + " -s",
    cwd=downloading_path_for_given_request,
    shell=True,
    check=True,
    text=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
) 

# -s option to run the script without credential | cwd is the directory of the script

## Store the log ##

with open(downloading_path_for_given_request + "/log.txt", "w") as log_file:

    log_file.write(download_process.stdout)

## Move the files to a more sustainable folder

After having downloaded the files for one model.variant couple, we would like the files to be stored in a place on our server that is known to not be cleaned often. For that, the user needs to define the variable *download_dir* that is a path.

### Create the folder at the download_dir path

Since we do a fresh download, we clean up the folder at the download_dir path and make a new one.

In [15]:
# ================ MAKE A FRESH DATA SAVING FOLDER ================ #

### EXTRACT THE PATH OF THE DIRECTORY IN WHICH LIES OUR SAVING DIRECTORY ###

download_dir_directory_path = os.path.dirname(download_dir)

### EXTRACT THE NAME OF THE SAVING DIRECTORY  ###

name_of_download_dir = os.path.basename(download_dir)

### USE OF THE CREATE DIR FUNCTION ###

download_dir = create_dir(download_dir_directory_path,  name_of_download_dir)

Pre-existing folder at /data/lgiboni/downloaded-cmip6-datadir-example removed
New folder at /data/lgiboni/downloaded-cmip6-datadir-example created


### Move the files from python

To move the model.variant folder, we will use the *mv_downloaded_data* function.

In [16]:
# ================ MOVING THE DOWNLOADED FILES TO download_dir PATH ================ #

### GET THE MODEL.VARIANT FOLDER ###

model_variant_folder_path = os.path.dirname(downloading_path_for_given_request)

### RUNNING THE MV BASH COMMAND FROM PYTHON ###

subprocess.run(
            "mv {} {}".format(model_variant_folder_path, download_dir), shell=True, check=True
        )
    
print("The {} folder was moved to the path {}".format(model_variant_name, download_dir))

The IPSL-CM6A-LR.r1i1p1f1 folder was moved to the path /data/lgiboni/downloaded-cmip6-datadir-example
